In [75]:
from collections import deque
from math import inf



# Funkcija za pronalaženje svih parova udaljenosti
def all_pairs_distance():
    n = len(graph)
    distance = [[inf] * n for _ in range(n)]

    for start in range(1, n + 1):  # Čvorovi numerisani od 1
        visited = [False] * n
        d_local = [inf] * n
        d_local[start - 1] = 0  # Indeksiranje za dist sa 1
        q = deque([start])
        visited[start - 1] = True

        while q:
            current = q.popleft()
            if current in graph:  # Provera da li je čvor u grafu
                for neighbor in graph[current]:
                    if not visited[neighbor - 1]:  # Indeksiranje za susede
                        visited[neighbor - 1] = True
                        q.append(neighbor)
                        d_local[neighbor - 1] = d_local[current - 1] + 1

        for i in range(n):
            distance[start - 1][i] = d_local[i]
            distance[i][start - 1] = d_local[i]

    return distance



In [76]:
from collections import deque
from math import inf

def BFF(n, graph, d):
    # Inicijalizacija
    first_vertex = 1  # Prvi čvor je numerisan sa 1
    s = [first_vertex]
    q = deque()
    
    # Udaljenosti do početnog čvora (koristimo d, pretpostavljamo da je d globalna matrica udaljenosti)
    dist = [d[i][0] for i in range(n)]
    
    # Lista za praćenje spaljenih čvorova
    C = [0] * n
    number_of_burned_vertices = 1
    q.append(first_vertex)
    C[first_vertex - 1] = 1  # Adjust index for C

    # Glavna petlja dok se ne spale svi čvorovi
    while number_of_burned_vertices < n:
        q_size = len(q)
        for _ in range(q_size):
            v = q.popleft()
            for neighbor in graph[v]:  # Pristupamo grafu sa v
                if C[neighbor - 1] == 0:  # Adjust index for C
                    q.append(neighbor)
                    C[neighbor - 1] = 1  # Adjust index for C
                    number_of_burned_vertices += 1

        # Pronalaženje najudaljenijeg čvora
        farthest_vertex = 1  # Prvi čvor je numerisan sa 1
        larger_distance = 0
        for i in range(n):
            if dist[i] > larger_distance:
                larger_distance = dist[i]
                farthest_vertex = i + 1  # Adjust index for 1-based numbering

        # Dodavanje najudaljenijeg čvora u red i ažuriranje
        q.append(farthest_vertex)
        s.append(farthest_vertex)
        C[farthest_vertex - 1] = 1  # Adjust index for C
        number_of_burned_vertices += 1

        # Ažuriranje udaljenosti
        for i in range(n):
            if d[i][farthest_vertex - 1] < dist[i]:  # Adjust index for d
                dist[i] = d[i][farthest_vertex - 1]  # Adjust index for d

    return s


In [77]:
def valid_burning_sequence(s, n, d):
    counter = 0
    for i in range(n):
        b = len(s)
        for j in range(b):
            if d[i][s[j] - 1] <= b - (j + 1):  # Adjust for 1-based indexing
                counter += 1
                break
    if counter == n:
        return True
    else:
        return False


In [78]:
# Pretpostavljamo da je `d` globalna promenljiva, definisana kao lista lista
# d = [[...], [...], ...]  # Definišite ovu matricu pre poziva funkcij



def Gr(n, k):
    s = []  # Lista koja će sadržavati rezultat
    max_value = 0
    max_value_vertex = 0
    C = [0] * n  # Lista veličine n, inicijalizovana na 0

    # Kreiranje matrice A
    A = []
    for i in range(n):
        a = []
        for j in range(n):
            if d[i][j] <= k - 1:
                a.append(j)
        A.append(a)

    # Glavni deo algoritma
    for r in range(k - 1, -1, -1):
        if r < k - 1:
            for i in range(n):
                j = 0
                while j < len(A[i]):
                    if d[i][A[i][j]] > r or C[A[i][j]] == 1:
                        # Uklanja čvor u O(1)
                        A[i][j] = A[i][-1]  # Zamenjuje trenutni element poslednjim
                        A[i].pop()  # Uklanja poslednji element
                    else:
                        j += 1

        max_value = 0
        max_value_vertex = 0

        for i in range(n):
            if len(A[i]) > max_value:
                max_value = len(A[i])
                max_value_vertex = i

        for j in A[max_value_vertex]:
            C[j] = 1

        s.append(max_value_vertex)

    return s


In [79]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph


In [80]:
from collections import deque
from math import inf

def BFF(n, graph, d):
    first_vertex = 1  # Prvi čvor je numerisan sa 1
    s = [first_vertex]
    q = deque()
    dist = [d[i][0] for i in range(n)]
    C = [0] * n
    number_of_burned_vertices = 1
    q.append(first_vertex)
    C[first_vertex - 1] = 1  # Adjust index for C

    while number_of_burned_vertices < n:
        q_size = len(q)
        for _ in range(q_size):
            v = q.popleft()
            for neighbor in graph[v]:
                if C[neighbor - 1] == 0:  # Adjust index for C
                    q.append(neighbor)
                    C[neighbor - 1] = 1  # Adjust index for C
                    number_of_burned_vertices += 1

        farthest_vertex = 1  # Prvi čvor je numerisan sa 1
        larger_distance = 0
        for i in range(n):
            if dist[i] > larger_distance:
                larger_distance = dist[i]
                farthest_vertex = i + 1  # Adjust index for 1-based numbering

        q.append(farthest_vertex)
        s.append(farthest_vertex)
        C[farthest_vertex - 1] = 1  # Adjust index for C
        number_of_burned_vertices += 1

        for i in range(n):
            if d[i][farthest_vertex - 1] < dist[i]:  # Adjust index for d
                dist[i] = d[i][farthest_vertex - 1]  # Adjust index for d

    return s

def multiple_BFF_sequences(num_sequences, n, graph, d):
    sequences = []
    for _ in range(num_sequences):
        sequence = BFF(n, graph, d)
        sequences.append(sequence)
    return sequences

# Pretpostavke:
# - `graph` je dict sa čvorovima numerisanim od 1
# - `d` je globalna matrica udaljenosti




In [81]:
#filename= "graf.mtx"  
filename = "grafovi/manji/karate.mtx"

graph = load_graph_from_mtx(filename)

In [82]:
d = all_pairs_distance()
best_burning_sequence = BFF(len(graph), graph, d);
print(best_burning_sequence)
low  = (len(best_burning_sequence)+2)
high = len(best_burning_sequence)-1;
n = len(graph)
import math
best_burning_sequence_size = len(best_burning_sequence)

k = math.floor((high + low) / 2)


[1, 15, 10, 16]


In [83]:
while low <= high:
    k = math.floor((high + low) / 2)
    s = Gr(n,k)
    if valid_burning_sequence(s, n, d):
        high = k-1;
        if len(s) <= best_burning_sequence_size:
            best_burning_sequence_size = len(s);
            best_burning_sequence = s;
        else:
            low = k+1;
        

In [84]:
print(s)


[0, 0, 0, 0]


In [87]:
import random
from collections import deque
from math import inf

def BFF(n, graph, d):
    # Nasumičan početni čvor
    first_vertex = random.randint(1, n)  # Nasumično biranje početnog čvora
    s = [first_vertex]
    q = deque()
    
    dist = [d[i][0] for i in range(n)]
    C = [0] * n
    number_of_burned_vertices = 1
    q.append(first_vertex)
    C[first_vertex - 1] = 1  # Adjust index for C

    while number_of_burned_vertices < n:
        q_size = len(q)
        # Nasumično obrađujemo čvorove u redu
        for _ in range(q_size):
            v = q.popleft()
            neighbors = list(graph[v])
            random.shuffle(neighbors)  # Nasumično raspoređivanje suseda
            for neighbor in neighbors:
                if C[neighbor - 1] == 0:  # Adjust index for C
                    q.append(neighbor)
                    C[neighbor - 1] = 1  # Adjust index for C
                    number_of_burned_vertices += 1

        # Nasumično pronalaženje najudaljenijeg čvora
        farthest_vertex = 1
        larger_distance = 0
        for i in range(n):
            if dist[i] > larger_distance:
                larger_distance = dist[i]
                farthest_vertex = i + 1  # Adjust index for 1-based numbering

        q.append(farthest_vertex)
        s.append(farthest_vertex)
        C[farthest_vertex - 1] = 1  # Adjust index for C
        number_of_burned_vertices += 1

        for i in range(n):
            if d[i][farthest_vertex - 1] < dist[i]:  # Adjust index for d
                dist[i] = d[i][farthest_vertex - 1]  # Adjust index for d

    return s

def multiple_BFF_sequences(num_sequences, n, graph, d):
    sequences = []
    for _ in range(num_sequences):
        sequence = BFF(n, graph, d)
        sequences.append(sequence)
    return sequences

# Pretpostavke:
# - `graph` je dict sa čvorovima numerisanim od 1
# - `d` je globalna matrica udaljenosti

# Primer poziva funkcije
num_sequences = 20  # Broj sekvenci koje želimo da dobijemo
burning_sequences = multiple_BFF_sequences(num_sequences, n, graph, d)
print(burning_sequences)


[[19, 15, 10, 16, 17], [6, 15, 10, 16], [34, 15, 10, 16], [21, 15, 10, 16, 17], [26, 15, 10, 16], [13, 15, 10, 16], [14, 15, 10, 16], [21, 15, 10, 16, 17], [22, 15, 10, 16], [23, 15, 10, 16, 17], [19, 15, 10, 16, 17], [21, 15, 10, 16, 17], [11, 15, 10, 16], [19, 15, 10, 16, 17], [27, 15, 10, 16, 17], [24, 15, 10, 16, 17], [16, 15, 10, 16, 17], [2, 15, 10, 16], [8, 15, 10, 16], [3, 15, 10, 16]]
